In [1]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
import numpy as np
import pickle
from matplotlib.patches import Rectangle

In [2]:
try:
    scope.dis()
    target.dis()
except Exception:
    pass
    

scope = cw.scope()
target = cw.target(scope)
scope.default_setup()
cw.program_target(scope, cw.programmers.STM32FProgrammer, "./live-CWNANO.hex")



See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5051 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5051 bytes


In [3]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk


scope.adc.samples = 1000

def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.simpleserial_write('m', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return np.array(trace)

def plot(trace):
    plt.plot(trace)
    plt.show()
    
def sub_plot(trc_arrs):
    plt_no = len(trc_arrs)
    fig, ax = plt.subplots(plt_no,sharex=False)
    for i, obj in enumerate(ax):
        obj.plot(trc_arrs[i])
        obj.set_title(i)
    
        
    plt.show()

In [7]:
n = 1
msg = bytearray([23,74,241,90])
target.simpleserial_write('a', msg)
reply = target.simpleserial_read('r',n)
lis = [int(b) for b in reply]
print(lis)
msg = bytearray([23^255,74^255,241^255,255^90])
target.simpleserial_write('a', msg)
reply = target.simpleserial_read('r',n)
lis = [int(b) for b in reply]
print(lis)

[1]
[0]


In [8]:
for i in [23,74,241,90]:
    print(chr(i))


J
ñ
Z


In [9]:
plot(trace)
# sub_plot([trace, trace])

In [8]:
np.savetxt("live.csv", trace, delimiter=",")